# Shapley 値推定精度比較実験（Kernel SHAP・Optimized Kernel SHAP・Leverage SHAP）

インポート

In [1]:
import numpy as np
import pandas as pd
import shap  # データセット読み込み & Optimized Kernel SHAP
import xgboost as xgb  # 決定木モデル（Tree SHAP 用）

np.random.seed(0)  # 再現性確保


c:\Users\shoei\miniconda3\envs\levshap_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


データセット読込 → XGBoost 学習 → 真の Shapley 値取得 (Tree SHAP)

In [2]:
# Prepare a dictionary to hold dataset info and results
datasets_info = {
    "iris": {"loader": shap.datasets.iris, "n_points": None, "task": "classification"},
    "california": {"loader": shap.datasets.california, "n_points": 1000, "task": "regression"},
    "diabetes": {"loader": shap.datasets.diabetes, "n_points": None, "task": "regression"},
    "adult": {"loader": shap.datasets.adult, "n_points": 1000, "task": "classification"},
    "correlated": {"loader": shap.datasets.corrgroups60, "n_points": 1000, "task": "regression"},
    "independent": {"loader": shap.datasets.independentlinear60, "n_points": 1000, "task": "regression"},
    "nhanes": {"loader": shap.datasets.nhanesi, "n_points": 1000, "task": "regression"},
    "communities": {"loader": shap.datasets.communitiesandcrime, "n_points": 1000, "task": "regression"},
}

# Storage for ground truth Shapley values and models
ground_truth_phi = {}
models = {}

X_full = {}  # 全行
x_exp_dict = {}  # 説明対象の 1 行（Series）

for name, info in datasets_info.items():
    # Load dataset
    if info["n_points"] is not None:
        X, y = info["loader"](n_points=info["n_points"])
    else:
        X, y = info["loader"]()  # load full dataset if n_points not specified
    # Ensure X is a DataFrame and y is an array or Series
    X = pd.DataFrame(X)  # (some shap loaders return np.array, ensure DataFrame)
    y = pd.Series(y)

    # Use the first data point as the instance to explain
    x_exp = X.iloc[0]
    x_exp_dict[name] = X.iloc[0]   # ★ 保存
    X_full[name] = X               # ★ 保存
    X_train = X.iloc[1:]
    y_train = y.iloc[1:]

    # Train XGBoost model
    if info["task"] == "classification":
        # Determine number of classes
        num_classes = len(np.unique(y_train))
        if num_classes > 2:
            model = xgb.XGBClassifier(eval_metric="logloss")
        else:
            model = xgb.XGBClassifier(eval_metric="logloss")
        model.fit(X_train, y_train)
    else:
        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)

    models[name] = model

    # Compute ground truth Shapley values using Tree SHAP
    explainer = shap.TreeExplainer(model)
    # Get shap values for the single instance (as a 1x n_features input)
    if info["task"] == "classification" and len(np.unique(y_train)) > 2:
        # For multi-class, TreeExplainer.shap_values returns a list of arrays (one per class)
        phi = explainer.shap_values(pd.DataFrame([x_exp]))
        # We take the Shapley values for the predicted class of this instance
        pred_class = model.predict(pd.DataFrame([x_exp]))[0]
        phi = phi[pred_class]  # shape (n_features,)
    else:
        # For regression or binary classification (TreeExplainer returns one array for binary)
        phi = explainer.shap_values(pd.DataFrame([x_exp]))[0]
    ground_truth_phi[name] = np.array(phi)
    print(f"{name.capitalize()} dataset: features = {X.shape[1]}, training samples = {X_train.shape[0]}.")


Iris dataset: features = 4, training samples = 149.
California dataset: features = 8, training samples = 999.
Diabetes dataset: features = 10, training samples = 441.
Adult dataset: features = 12, training samples = 999.
Correlated dataset: features = 60, training samples = 999.
Independent dataset: features = 60, training samples = 999.
Nhanes dataset: features = 79, training samples = 999.
Communities dataset: features = 102, training samples = 999.


 Kernel SHAP (Monte Carlo sampling + weighted regression)の実装

In [3]:
def kernel_shap_estimate(baseline, explicand, model_fn, num_samples):
    """
    Estimate Shapley values using Kernel SHAP (Monte Carlo sampling + weighted regression).
    :param baseline: 1D numpy array of baseline feature values (for "missing" features).
    :param explicand: 1D numpy array of the instance's feature values (to explain).
    :param model_fn: function that takes a 2D array of inputs and returns model outputs.
                     For classification, model_fn should return the raw prediction (log-odds or probability) for the class of interest.
    :param num_samples: number of subset samples to draw (excluding the empty and full subsets which are always used).
    :return: numpy array of estimated Shapley values (length = number of features).
    """
    n = explicand.shape[0]
    # Prepare arrays for design matrix and target values
    X_mat = []
    y_vec = []
    weights = []
    
    # Ensure empty set and full set are included
    empty_mask = np.zeros(n, dtype=int)
    full_mask = np.ones(n, dtype=int)
    X_mat.append(empty_mask)
    X_mat.append(full_mask)
    # Model evaluations for empty and full
    y_empty = model_fn(np.array([baseline]))[0]
    y_full = model_fn(np.array([explicand]))[0]
    y_vec.append(y_empty)
    y_vec.append(y_full)
    # Assign a very large weight to empty and full to treat them as constraints in the regression
    weights.append(10**6)
    weights.append(10**6)
    
    # Random generator
    rng = np.random.default_rng()
    for _ in range(num_samples):
        mask = rng.integers(0, 2, size=n, endpoint=False)  # random 0/1 mask
        if mask.sum() == 0 or mask.sum() == n:
            continue  # skip if accidentally got all-0 or all-1 (already included)
        # Prepare the input sample: baseline for 0 features, explicand for 1 features
        sample = baseline.copy()
        sample[mask == 1] = explicand[mask == 1]
        # Evaluate model
        y_val = model_fn(np.array([sample]))[0]
        X_mat.append(mask.astype(int))
        y_vec.append(y_val)
        # Kernel weight for this subset
        k = mask.sum()
        # Weight formula: (n-1) / [C(n, k) * k * (n-k)] (Kernel SHAP weighting)
        # Use math.comb for binomial coefficient
        W = (n - 1) / ( (np.math.comb(n, k)) * k * (n - k) )
        weights.append(W)
    
    X_mat = np.array(X_mat)  # shape (m_total, n)
    y_vec = np.array(y_vec)
    weights = np.array(weights)
    
    # Use weighted least squares to solve for phi (Shapley values).
    # We center the outputs by subtracting the model output for empty set (which will serve as phi0).
    phi0 = y_vec[0]  # y_empty
    y_centered = y_vec - phi0
    # Apply weights: multiply each row by sqrt(weight)
    W_sqrt = np.sqrt(weights)
    Xw = X_mat * W_sqrt[:, None]
    yw = y_centered * W_sqrt
    # Solve least squares
    phi, *_ = np.linalg.lstsq(Xw, yw, rcond=None)
    return phi


Optimized Kernel SHAP (SHAP Library)の実装

In [4]:
def optimized_kernel_shap_estimate(background_data, explicand, model_fn, num_samples):
    """
    Estimate Shapley values using SHAP's optimized KernelExplainer.
    :param background_data: DataFrame or numpy array for background (reference) distribution of features.
    :param explicand: 1D numpy array of the instance's feature values to explain.
    :param model_fn: function that returns the model output of interest (should be vectorized).
    :param num_samples: number of evaluations to use in KernelExplainer (nsamples parameter).
    :return: numpy array of estimated Shapley values.
    """
    explainer = shap.KernelExplainer(model_fn, background_data)
    phi = explainer.shap_values(explicand, nsamples=num_samples)
    phi = np.array(phi)
    # KernelExplainer returns a single array for regression or binary classification,
    # but returns a list of arrays for multi-class. We handle both:
    if isinstance(phi, list):
        # If it's multi-class, we assume model_fn was set to output only the class of interest,
        # so shap_values returns a list of length 1.
        phi = phi[0]
    return phi


 Leverage SHAP (Leverage Score Sampling)を近似的に実装。（元のリポジトリの実装とは異なる）

In [5]:
import math
import itertools


def leverage_shap_estimate(baseline, explicand, model_fn, num_samples):
    """
    Estimate Shapley values using Leverage SHAP (approximate leverage score sampling).
    :param baseline: 1D numpy array of baseline feature values.
    :param explicand: 1D numpy array of instance feature values.
    :param model_fn: function that returns model output of interest.
    :param num_samples: total number of subset samples to draw.
    :return: numpy array of estimated Shapley values.
    """
    n = explicand.shape[0]
    X_mat = []
    y_vec = []
    weights = []
    # Include empty and full as constraints (with high weight)
    empty_mask = np.zeros(n, dtype=int)
    full_mask = np.ones(n, dtype=int)
    X_mat.append(empty_mask)
    X_mat.append(full_mask)
    y_empty = model_fn(np.array([baseline]))[0]
    y_full = model_fn(np.array([explicand]))[0]
    y_vec.append(y_empty)
    y_vec.append(y_full)
    weights.append(10**6)
    weights.append(10**6)
    # Determine how many subsets to sample for each size 1..n-1
    size_probs = np.array([0 if (k == 0 or k == n) else 1 / (k * (n - k)) for k in range(n + 1)], dtype=float)
    size_probs = size_probs / size_probs.sum()
    # Draw number of samples for each subset size
    samples_per_size = {k: 0 for k in range(1, n)}
    remaining = num_samples
    # Use multinomial draw for how many subsets of each size
    draws = np.random.multinomial(num_samples, [size_probs[k] for k in range(n + 1)])
    for k in range(1, n):
        samples_per_size[k] = draws[k]
    # Now sample subsets for each size without replacement
    for k, m_k in samples_per_size.items():
        if m_k <= 0:
            continue
        # All possible subsets of size k (could be large if n is big; we sample without replacement)
        # If total combinations C(n, k) is less than m_k, we'll take all.
        # Otherwise, randomly choose m_k subsets of size k.
        if m_k >= math.comb(n, k):
            subsets_k = list(itertools.combinations(range(n), k))
        else:
            subsets_k = list(itertools.combinations(range(n), k))
            subsets_k = np.array(subsets_k, dtype=object)
            # choose m_k unique subsets
            subset_indices = np.random.choice(len(subsets_k), size=m_k, replace=False)
            subsets_k = subsets_k[subset_indices]
        for comb_idx in subsets_k:
            mask = np.zeros(n, dtype=int)
            mask[list(comb_idx)] = 1
            sample = baseline.copy()
            sample[mask == 1] = explicand[mask == 1]
            y_val = model_fn(np.array([sample]))[0]
            X_mat.append(mask)
            y_vec.append(y_val)
            # We use the same Kernel SHAP weight for consistency
            k_size = mask.sum()
            W = (n - 1) / ((math.comb(n, k_size)) * k_size * (n - k_size))
            weights.append(W)
    X_mat = np.array(X_mat)
    y_vec = np.array(y_vec)
    weights = np.array(weights)
    # Solve weighted least squares (same as in kernel_shap_estimate)
    phi0 = y_vec[0]
    y_centered = y_vec - phi0
    W_sqrt = np.sqrt(weights)
    Xw = X_mat * W_sqrt[:, None]
    yw = y_centered * W_sqrt
    phi, *_ = np.linalg.lstsq(Xw, yw, rcond=None)
    return phi


Run Experiments and Compare Shapley Value Estimation Errors

In [6]:
# Settings for sample sizes and trials
sample_multiples = [5, 10, 20]  # correspond to 5n, 10n, 20n
num_trials = 100  # number of random runs for each setting

# Prepare a dataframe to collect median L2 errors
results_df = pd.DataFrame(index=pd.MultiIndex.from_product(
    [datasets_info.keys(), [f"{m}n" for m in sample_multiples]]),
    columns=["Kernel SHAP", "Optimized Kernel SHAP", "Leverage SHAP"]
)

# Evaluate each dataset
for name in datasets_info.keys():
    # Prepare baseline vector as the mean of training data features (for missing feature values)
    X_train = pd.DataFrame(models[name].get_booster().feature_names, columns=None)
    # If the model doesn't keep feature_names, use the original X training data we stored:
    # (We can retrieve baseline from X used earlier in ground_truth_phi calculation)
    X_train = X_train if not X_train.empty else pd.DataFrame(ground_truth_phi[name]).T.iloc[0:0]  # fallback (adjust as needed)

# We will use the training data as background for KernelExplainer and derive baseline from it
baseline_vecs = {}
background_data = {}

# (Re-run the data loading loop to also store baseline and background data)
for name, info in datasets_info.items():
    if info["n_points"] is not None:
        X, y = info["loader"](n_points=info["n_points"])
    else:
        X, y = info["loader"]()
    X = pd.DataFrame(X)
    y = pd.Series(y)
    x_exp = X.iloc[0]  # instance to explain
    X_train = X.iloc[1:]
    y_train = y.iloc[1:]
    # Train model (same as before)
    if info["task"] == "classification":
        num_classes = len(np.unique(y_train))
        model = xgb.XGBClassifier(eval_metric="logloss") if num_classes <= 2 else xgb.XGBClassifier(eval_metric="mlogloss")
        model.fit(X_train, y_train)
    else:
        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)
    models[name] = model
    explainer = shap.TreeExplainer(model)
    if info["task"] == "classification" and len(np.unique(y_train)) > 2:
        phi_all = explainer.shap_values(pd.DataFrame([x_exp]))
        pred_class = model.predict(pd.DataFrame([x_exp]))[0]
        ground_truth_phi[name] = np.array(phi_all[pred_class])
    else:
        ground_truth_phi[name] = np.array(explainer.shap_values(pd.DataFrame([x_exp]))[0])
    # Store baseline vector (mean of training features) and background data
    baseline_vecs[name] = X_train.mean().values
    background_data[name] = X_train


In [7]:
# Define sample size values in terms of multiples of n (number of features)
sample_multiples = [5, 10, 20]
num_trials = 30  # use 30 for demo (use 100 for full experiment)

# Results storage: a multi-index DataFrame [dataset, m] x [method]
results = pd.DataFrame(
    index=pd.MultiIndex.from_product([datasets_info.keys(), [f"{m}n" for m in sample_multiples]]),
    columns=["Kernel SHAP", "Optimized Kernel SHAP", "Leverage SHAP"],
)

for name, info in datasets_info.items():
    n = len(baseline_vecs[name])  # number of features
    X_dataset = X_full[name]  # ★ データフレームを取り出す
    x_exp = x_exp_dict[name]  # ★ 説明対象 1 行を取り出す
    # Define model function for shap computations:
    if info["task"] == "classification" and models[name].n_classes_ > 2:
        pred_class = models[name].predict(pd.DataFrame([x_exp]))[0]

        def model_fn(X_batch, model=models[name], c=pred_class):
            margin = model.predict(pd.DataFrame(X_batch), output_margin=True)
            return margin[:, c]
    elif info["task"] == "classification":

        def model_fn(X_batch, model=models[name]):
            return model.predict(pd.DataFrame(X_batch), output_margin=True)
    else:

        def model_fn(X_batch, model=models[name]):
            return model.predict(pd.DataFrame(X_batch))

    baseline = baseline_vecs[name]
    explicand = x_exp.values  # ★ 対象行の値

    for m_mult in sample_multiples:
        m_val = m_mult * n
        errors_kernel = []
        errors_opt = []
        errors_leverage = []
        for t in range(num_trials):
            # Kernel SHAP (Monte Carlo)
            phi_kernel = kernel_shap_estimate(baseline, explicand, model_fn, num_samples=m_val)
            err_kernel = np.linalg.norm(phi_kernel - ground_truth_phi[name])
            errors_kernel.append(err_kernel)
            # Optimized Kernel SHAP (shap.KernelExplainer)
            phi_opt = optimized_kernel_shap_estimate(background_data[name], explicand, model_fn, num_samples=m_val)
            err_opt = np.linalg.norm(phi_opt - ground_truth_phi[name])
            errors_opt.append(err_opt)
            # Leverage SHAP
            phi_leverage = leverage_shap_estimate(baseline, explicand, model_fn, num_samples=m_val)
            err_lev = np.linalg.norm(phi_leverage - ground_truth_phi[name])
            errors_leverage.append(err_lev)
        # Compute median errors
        median_kernel = float(np.median(errors_kernel))
        median_opt = float(np.median(errors_opt))
        median_leverage = float(np.median(errors_leverage))
        results.loc[(name, f"{m_mult}n"), "Kernel SHAP"] = median_kernel
        results.loc[(name, f"{m_mult}n"), "Optimized Kernel SHAP"] = median_opt
        results.loc[(name, f"{m_mult}n"), "Leverage SHAP"] = median_leverage
        print(
            f"{name.capitalize()} (m={m_mult}n): Kernel={median_kernel:.4f}, Optimized={median_opt:.4f}, Leverage={median_leverage:.4f}"
        )


ValueError: feature_names mismatch: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] ['0', '1', '2', '3']
expected petal length (cm), petal width (cm), sepal length (cm), sepal width (cm) in input data
training data did not have the following fields: 3, 0, 1, 2

In [ ]:
# Display the median L2 errors for each dataset and each method (for m = 5n, 10n, 20n)
for name in datasets_info.keys():
    print(f"\n**{name.capitalize()} dataset (n={len(baseline_vecs[name])} features)**")
    for m_mult in sample_multiples:
        med_kernel = results.loc[(name, f"{m_mult}n"), "Kernel SHAP"]
        med_opt = results.loc[(name, f"{m_mult}n"), "Optimized Kernel SHAP"]
        med_lev = results.loc[(name, f"{m_mult}n"), "Leverage SHAP"]
        print(
            f"  m = {m_mult}n:  Kernel SHAP = {med_kernel:.4f},  Optimized Kernel SHAP = {med_opt:.4f},  Leverage SHAP = {med_lev:.4f}"
        )



**Iris dataset (n=4 features)**
  m = 5n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 10n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 20n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan

**California dataset (n=8 features)**
  m = 5n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 10n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 20n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan

**Diabetes dataset (n=10 features)**
  m = 5n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 10n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 20n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan

**Adult dataset (n=12 features)**
  m = 5n:  Kernel SHAP = nan,  Optimized Kernel SHAP = nan,  Leverage SHAP = nan
  m = 10n:  Kernel SHAP = nan,  Optimiz